In [4]:
# Import the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm  import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
!pip install scikit-learn


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Load Dataset

titanic = pd.read_csv(r"D:\Python with Data Science\Datasets\titanic.csv")

In [11]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
titanic.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [ ]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Compare survival across Age, SibSp, Parch, Fare

In [ ]:
pd.pivot_table(titanic, index = 'Survived', values = ['Age', 'SibSp', 'Parch', 'Fare'], aggfunc ='mean')

,Age,Fare,Parch,SibSp
Survived,,,,
0,30.626179,22.117887,0.329690,0.553734
1,28.343690,48.395408,0.464912,0.473684


In [ ]:
titanic.groupby('Survived')[['Age', 'Fare']].mean()

,Age,Fare
Survived,,
0,30.626179,22.117887
1,28.343690,48.395408


In [ ]:
pd.pivot_table(titanic, index ='Survived', columns = 'Pclass', values='Ticket', aggfunc = 'count')

Pclass,1,2,3
Survived,,,
0,80,97,372
1,136,87,119


In [ ]:
pd.pivot_table(titanic, index ='Survived', columns = 'Sex', values='Ticket', aggfunc = 'count')

Sex,female,male
Survived,,
0,81,468
1,233,109


In [ ]:
pd.pivot_table(titanic, index ='Survived', columns = 'Embarked', values='Ticket', aggfunc = 'count')

Embarked,C,Q,S
Survived,,,
0,75,47,427
1,93,30,217


In [ ]:
pd.pivot_table(titanic, index ='Survived', columns = 'Parch', values='Ticket', aggfunc = 'count')

Parch,0,1,2,3,4,5,6
Survived,,,,,,,
0,445.0,53.0,40.0,2.0,4.0,4.0,1.0
1,233.0,65.0,40.0,3.0,NaN,1.0,NaN


In [ ]:
# Feature Engineering  - Create new meaningful data from existing data

1) Cabin - Simplify cabins (evaluated if cabin letter (cabin_letter) or the purchase of tickets across multiple cabins (cabin_multiple) impacted survival)

2) Tickets - Do different ticket types impact survival rates?

3) Does a person's title relate to survival rates?

In [ ]:
titanic['Cabin'].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [ ]:
titanic['Cabin_Letter'] =  titanic.Cabin.apply(lambda x: str(x)[0])

In [ ]:
titanic['Cabin_Count'] = titanic.Cabin.apply(lambda x : 0 if pd.isna(x) else len(x.split(' ')))

In [ ]:
pd.pivot_table(titanic, index ='Survived', columns = 'Cabin_Letter', values='Ticket', aggfunc = 'count')

Cabin_Letter,A,B,C,D,E,F,G,T,n
Survived,,,,,,,,,
0,8.0,12.0,24.0,8.0,8.0,5.0,2.0,1.0,481.0
1,7.0,35.0,35.0,25.0,24.0,8.0,2.0,NaN,206.0


In [ ]:
pd.pivot_table(titanic, index ='Survived', columns = 'Cabin_Count', values='Ticket', aggfunc = 'count')

Cabin_Count,0,1,2,3,4
Survived,,,,,
0,481.0,58.0,7.0,3.0,NaN
1,206.0,122.0,9.0,3.0,2.0


In [ ]:
titanic.drop(columns = ['PassengerId', 'Cabin'], inplace = True)

In [ ]:
titanic['Num_ticket'] = titanic.Ticket.apply(lambda x:  1 if x.isnumeric() else 0)

In [ ]:
pd.pivot_table(titanic, index ='Survived', columns = 'Num_ticket', values='Ticket', aggfunc = 'count')

Num_ticket,0,1
Survived,,
0,142,407
1,88,254


In [ ]:
titanic['Title'] = titanic.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [ ]:
pd.pivot_table(titanic, index ='Survived', columns = 'Title', values='Ticket', aggfunc = 'count')

Title,Capt,Col,Don,Dr,Jonkheer,Lady,Major,Master,Miss,Mlle,Mme,Mr,Mrs,Ms,Rev,Sir,the Countess
Survived,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,4.0,1.0,NaN,1.0,17.0,55.0,NaN,NaN,436.0,26.0,NaN,6.0,NaN,NaN
1,NaN,1.0,NaN,3.0,NaN,1.0,1.0,23.0,127.0,2.0,1.0,81.0,99.0,1.0,NaN,1.0,1.0


In [ ]:
titanic.drop(columns = ['Name', 'Ticket'], inplace = True)

In [ ]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Survived      891 non-null    int64  
 1   Pclass        891 non-null    int64  
 2   Sex           891 non-null    object 
 3   Age           714 non-null    float64
 4   SibSp         891 non-null    int64  
 5   Parch         891 non-null    int64  
 6   Fare          891 non-null    float64
 7   Embarked      889 non-null    object 
 8   Cabin_Letter  891 non-null    object 
 9   Cabin_Count   891 non-null    int64  
 10  Num_ticket    891 non-null    int64  
 11  Title         891 non-null    object 
dtypes: float64(2), int64(6), object(4)
memory usage: 83.7+ KB


In [ ]:
# Missing Values

In [ ]:
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].mean())
titanic['Embarked'] = titanic['Embarked'].fillna(titanic['Embarked'].mode()[0])

In [ ]:
titanic.isnull().sum()

Survived        0
Pclass          0
Sex             0
Age             0
SibSp           0
Parch           0
Fare            0
Embarked        0
Cabin_Letter    0
Cabin_Count     0
Num_ticket      0
Title           0
dtype: int64

In [ ]:
# Preprocess Data
# Convert all Categorical data to numerical data
# Scale data using standard scaler

In [ ]:
X = titanic.iloc[:,1:]
Y = titanic.iloc[:,0]

In [ ]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin_Letter,Cabin_Count,Num_ticket,Title
0,3,male,22.0,1,0,7.2500,S,n,0,0,Mr
1,1,female,38.0,1,0,71.2833,C,C,1,0,Mrs
2,3,female,26.0,0,0,7.9250,S,n,0,0,Miss
3,1,female,35.0,1,0,53.1000,S,C,1,1,Mrs
4,3,male,35.0,0,0,8.0500,S,n,0,1,Mr


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        891 non-null    int8   
 1   Sex           891 non-null    object 
 2   Age           891 non-null    float64
 3   SibSp         891 non-null    int64  
 4   Parch         891 non-null    int64  
 5   Fare          891 non-null    float64
 6   Embarked      891 non-null    object 
 7   Cabin_Letter  891 non-null    object 
 8   Cabin_Count   891 non-null    int64  
 9   Num_ticket    891 non-null    int64  
 10  Title         891 non-null    object 
dtypes: float64(2), int64(4), int8(1), object(4)
memory usage: 70.6+ KB


In [ ]:
# get_dummies

In [ ]:
X = pd.get_dummies(X, dtype = 'int8')

In [ ]:
X.head()

,Pclass,Age,SibSp,Parch,Fare,Cabin_Count,Num_ticket,Sex_female,Sex_male,Embarked_C,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
0,3,22.0,1,0,7.2500,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,1,38.0,1,0,71.2833,1,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
2,3,26.0,0,0,7.9250,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,35.0,1,0,53.1000,1,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,3,35.0,0,0,8.0500,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Pclass              891 non-null    int8   
 1   Age                 891 non-null    float64
 2   SibSp               891 non-null    int64  
 3   Parch               891 non-null    int64  
 4   Fare                891 non-null    float64
 5   Cabin_Count         891 non-null    int64  
 6   Num_ticket          891 non-null    int64  
 7   Sex_female          891 non-null    int8   
 8   Sex_male            891 non-null    int8   
 9   Embarked_C          891 non-null    int8   
 10  Embarked_Q          891 non-null    int8   
 11  Embarked_S          891 non-null    int8   
 12  Cabin_Letter_A      891 non-null    int8   
 13  Cabin_Letter_B      891 non-null    int8   
 14  Cabin_Letter_C      891 non-null    int8   
 15  Cabin_Letter_D      891 non-null    int8   
 16  Cabin_Le

In [ ]:
X['Pclass'] = X['Pclass'].astype('int8')

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Pclass              891 non-null    int8   
 1   Age                 891 non-null    float64
 2   SibSp               891 non-null    int64  
 3   Parch               891 non-null    int64  
 4   Fare                891 non-null    float64
 5   Cabin_Count         891 non-null    int64  
 6   Num_ticket          891 non-null    int64  
 7   Sex_female          891 non-null    int8   
 8   Sex_male            891 non-null    int8   
 9   Embarked_C          891 non-null    int8   
 10  Embarked_Q          891 non-null    int8   
 11  Embarked_S          891 non-null    int8   
 12  Cabin_Letter_A      891 non-null    int8   
 13  Cabin_Letter_B      891 non-null    int8   
 14  Cabin_Letter_C      891 non-null    int8   
 15  Cabin_Letter_D      891 non-null    int8   
 16  Cabin_Le

In [ ]:
# Standard SCaling

In [ ]:
sc = StandardScaler()

In [ ]:
newX = sc.fit_transform(X)

In [ ]:
X.describe()

,Pclass,Age,SibSp,Parch,Fare,Cabin_Count,Num_ticket,Sex_female,Sex_male,Embarked_C,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208,0.267116,0.741863,0.352413,0.647587,0.188552,...,0.044893,0.204265,0.002245,0.001122,0.580247,0.140292,0.001122,0.006734,0.001122,0.001122
std,0.836071,13.002015,1.102743,0.806057,49.693429,0.547134,0.437855,0.477990,0.477990,0.391372,...,0.207186,0.403390,0.047351,0.033501,0.493796,0.347485,0.033501,0.081830,0.033501,0.033501
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,29.699118,0.000000,0.000000,14.454200,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,35.000000,1.000000,0.000000,31.000000,0.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,4.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
pd.DataFrame(newX, columns = X.columns).describe()

,Pclass,Age,SibSp,Parch,Fare,Cabin_Count,Num_ticket,Sex_female,Sex_male,Embarked_C,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
count,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,...,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,891.000000,8.910000e+02,8.910000e+02,8.910000e+02
mean,-8.772133e-17,2.232906e-16,4.386066e-17,5.382900e-17,3.987333e-18,-4.984166e-17,4.784800e-17,3.987333e-17,-1.156327e-16,-1.993666e-17,...,4.784800e-17,1.036707e-16,2.791133e-17,3.987333e-18,-7.974666e-18,-2.990500e-17,0.000000,1.594933e-17,3.987333e-18,3.987333e-18
std,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,...,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562,1.000562e+00,1.000562e+00,1.000562e+00
min,-1.566107e+00,-2.253155e+00,-4.745452e-01,-4.736736e-01,-6.484217e-01,-4.884829e-01,-1.695262e+00,-7.376951e-01,-1.355574e+00,-4.820427e-01,...,-2.168030e-01,-5.066553e-01,-4.743120e-02,-3.352008e-02,-1.175735e+00,-4.039621e-01,-0.033520,-8.233870e-02,-3.352008e-02,-3.352008e-02
25%,-3.693648e-01,-5.924806e-01,-4.745452e-01,-4.736736e-01,-4.891482e-01,-4.884829e-01,-1.695262e+00,-7.376951e-01,-1.355574e+00,-4.820427e-01,...,-2.168030e-01,-5.066553e-01,-4.743120e-02,-3.352008e-02,-1.175735e+00,-4.039621e-01,-0.033520,-8.233870e-02,-3.352008e-02,-3.352008e-02
50%,8.273772e-01,0.000000e+00,-4.745452e-01,-4.736736e-01,-3.573909e-01,-4.884829e-01,5.898793e-01,-7.376951e-01,7.376951e-01,-4.820427e-01,...,-2.168030e-01,-5.066553e-01,-4.743120e-02,-3.352008e-02,8.505317e-01,-4.039621e-01,-0.033520,-8.233870e-02,-3.352008e-02,-3.352008e-02
75%,8.273772e-01,4.079260e-01,4.327934e-01,-4.736736e-01,-2.424635e-02,-4.884829e-01,5.898793e-01,1.355574e+00,7.376951e-01,-4.820427e-01,...,-2.168030e-01,-5.066553e-01,-4.743120e-02,-3.352008e-02,8.505317e-01,-4.039621e-01,-0.033520,-8.233870e-02,-3.352008e-02,-3.352008e-02
max,8.273772e-01,3.870872e+00,6.784163e+00,6.974147e+00,9.667167e+00,6.826447e+00,5.898793e-01,1.355574e+00,7.376951e-01,2.074505e+00,...,4.612483e+00,1.973729e+00,2.108317e+01,2.983287e+01,8.505317e-01,2.475480e+00,29.832868,1.214496e+01,2.983287e+01,2.983287e+01


In [ ]:
newX, Y

(array([[ 0.82737724, -0.5924806 ,  0.43279337, ..., -0.0823387 ,
         -0.03352008, -0.03352008],
        [-1.56610693,  0.63878901,  0.43279337, ..., -0.0823387 ,
         -0.03352008, -0.03352008],
        [ 0.82737724, -0.2846632 , -0.4745452 , ..., -0.0823387 ,
         -0.03352008, -0.03352008],
        ...,
        [ 0.82737724,  0.        ,  0.43279337, ..., -0.0823387 ,
         -0.03352008, -0.03352008],
        [-1.56610693, -0.2846632 , -0.4745452 , ..., -0.0823387 ,
         -0.03352008, -0.03352008],
        [ 0.82737724,  0.17706291, -0.4745452 , ..., -0.0823387 ,
         -0.03352008, -0.03352008]]),
 0      0
 1      1
 2      1
 3      1
 4      0
       ..
 886    0
 887    1
 888    0
 889    1
 890    0
 Name: Survived, Length: 891, dtype: int64)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(newX, Y, test_size = 0.3)

In [ ]:
# Random Forest Classifier

In [ ]:
rfc =  RandomForestClassifier()
rfc.fit(xtrain,ytrain)
pred_rfc =rfc.predict(xtest)
print(classification_report(ytest, pred_rfc))

              precision    recall  f1-score   support

           0       0.80      0.87      0.84       151
           1       0.82      0.72      0.76       117

    accuracy                           0.81       268
   macro avg       0.81      0.80      0.80       268
weighted avg       0.81      0.81      0.80       268



In [ ]:
lr =  LogisticRegression()
lr.fit(xtrain,ytrain)
pred_lr =lr.predict(xtest)
print(classification_report(ytest, pred_lr))

              precision    recall  f1-score   support

           0       0.78      0.85      0.82       151
           1       0.79      0.69      0.74       117

    accuracy                           0.78       268
   macro avg       0.78      0.77      0.78       268
weighted avg       0.78      0.78      0.78       268



In [ ]:
sv =  SVC()
sv.fit(xtrain,ytrain)
pred_sv =sv.predict(xtest)
print(classification_report(ytest, pred_sv))

              precision    recall  f1-score   support

           0       0.80      0.85      0.83       151
           1       0.79      0.73      0.76       117

    accuracy                           0.80       268
   macro avg       0.80      0.79      0.79       268
weighted avg       0.80      0.80      0.80       268



In [ ]:
dtc =  DecisionTreeClassifier()
dtc.fit(xtrain,ytrain)
pred_dtc =dtc.predict(xtest)
print(classification_report(ytest, pred_dtc))

              precision    recall  f1-score   support

           0       0.75      0.80      0.78       151
           1       0.72      0.66      0.69       117

    accuracy                           0.74       268
   macro avg       0.74      0.73      0.73       268
weighted avg       0.74      0.74      0.74       268



In [ ]:
from sklearn.tree import export_text

In [ ]:
print(export_text(dtc))

|--- feature_32 <= -0.16
|   |--- feature_0 <= 0.23
|   |   |--- feature_35 <= 6.03
|   |   |   |--- feature_7 <= 0.31
|   |   |   |   |--- feature_3 <= 0.15
|   |   |   |   |   |--- feature_1 <= 1.33
|   |   |   |   |   |   |--- feature_1 <= 0.41
|   |   |   |   |   |   |   |--- feature_5 <= 0.43
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- feature_5 >  0.43
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_1 >  0.41
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_1 >  1.33
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_3 >  0.15
|   |   |   |   |   |--- class: 1
|   |   |   |--- feature_7 >  0.31
|   |   |   |   |--- feature_1 <= -2.09
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_1 >  -2.09
|   |   |   |   |   |--- feature_4 <= -0.12
|   |   |   |   |   |   |--- feature_1 <= 2.02
|   |   |   |   |   |   |   |--- feature_1 <= 0.56
|   |   |   |   |   |   